In [1]:
# Import Dependencies

from time import sleep
# from features import get_features
from mido import MidiFile
import pandas as pd

from datetime import date
from time import time, monotonic
from os import path, mkdir, listdir, environ
import subprocess


In [2]:
# Get Args, define evaluation groups

iterations = 2        # How many times each experiment should be repeated for the same parameters?
max_input_bars = 2    # What is the maximum lookback size for each model?
max_output_bars = 2   # What is the maximum output size for each model?
# sample_length = 32    # How long of a final composition should be generated?
sample_length = 2     # How long of a final composition should be generated?

client  = None        # Client Process
server  = None        # Server Process
expname = ''          # Experiment Name, for naming files

ip = "127.0.0.1"
port = 5005
port_in = 57120

modelname = None
checkpointname = None

evaluation_sets = [
  ('bach-chorales', [
    ('polyphony_rnn', 'polyphony_rnn'),
    ('pianoroll_rnn_nade', 'rnn-nade_attn'),
    ('rl_duet', 'rl_duet'),
  ]),
  ('piano-e-comp', [
    ('performance_rnn', 'performance_with_dynamics'),
    # ('music_transformer', 'performance_with_dynamics')
  ]),
  (None, [
    # ('melody_rnn', 'attention_rnn')
  ]),
  (None, [
    # ('remi', 'remi')
  ])
]
cols_gen = [ "model", "checkpoint", "primer", "out_file", "time", "in_len", "out_len" ]

In [3]:
# Folder Definitions
primerdir = ''
# outputdir = path.join(path.pardir,'output')
# datasetdir = path.join(path.pardir,'dataset') 
outputdir = path.join(path.curdir,'output')
datasetdir = path.join(path.curdir,'dataset')
basefoldername = str(date.today())
i = 0
while True:
    foldername = f'{basefoldername}-{i}'
    full_foldername = path.join(outputdir, foldername)

    if not path.exists(full_foldername):
      mkdir(full_foldername)
      break

    if not any(listdir(full_foldername)): break
    i = i + 1


# File Management
def get_filename(expname,index,primer=None):
  return path.normpath(
    f"{foldername}/{expname}-{index}"
    if primer is None else
    f"{foldername}/{expname}-{primer}-{index}")

def get_primer_filename(name):
  return path.join(primerdir,name)

def get_midi_filename(expname,index,primer=None):
  return path.join(outputdir, f'{get_filename(expname,index,primer)}.mid')

def get_pickle_filename(expname,index,primer=None):
  return path.join(outputdir, f'{get_filename(expname,index,primer)}.pkl')

def save_df(df, filename):
    # log(f'Saving dataframe to {filename}')
    df.to_pickle(filename)

def log(message):
    print(f'[batch:{expname}] {message}')

In [4]:
# Define Experiments

def generate_sample(primer, bars_input, bars_output):
  client.reset()
  client.load_bars(get_primer_filename(primer), bars_input)

  i = 0
  start_time = monotonic()
  while bars_input + i * bars_output < sample_length:
  # for b in range(bars_output):
    client.generate(bars_output, 'measures')
    i += 1
  gentime = monotonic() - start_time

  filename = get_filename('generation',i,f'{primer}-{bars_input}-{bars_output}-bars')
  client.save(filename)
  return filename, gentime

def run_generation(primer):
  return [[modelname, checkpointname, primer, *generate_sample(primer, bars_input, bars_output), bars_input, bars_output]
    for bars_input
    in range(1,max_input_bars)
    for bars_output
    in range(1,max_output_bars)
  ]

experiments = [
  # ('input_len', analysis_input_length),
  # ('output_len', analysis_output_length),
  ('generation', run_generation)
  ]

def experiment(name, fn):
  expname = name
  df = pd.DataFrame([], columns=cols_gen)

  for primer in primers:
    for i in range(iterations):
      out = fn(primer)
      print(out)
      df += pd.DataFrame(out, columns=cols_gen)
  print(df.head())
  return df

In [5]:
# Start Client, define model management functions
from pythonosc import udp_client, osc_server
from pythonosc.dispatcher import Dispatcher

class BatchClient(udp_client.SimpleUDPClient):
  def __init__(self, logger, ip, port_out, port_in):
    udp_client.SimpleUDPClient.__init__(self, ip, port_out, port_in)
    dispatcher = Dispatcher()
    dispatcher.map('/ok', lambda _: self.server.shutdown())
    self.server = osc_server.ThreadingOSCUDPServer((ip, port_in), dispatcher)
    self.log = logger

  def request(self, endpoint, args):
    self.log(f'{endpoint} {" ".join([str(a) for a in args])}')
    self.send_message(endpoint, args)

  def start(self):
    self.request('/start', [])

  def stop(self):
    self.request('/stop', [])

  def set(self,*args):
    self.request('/set', args)

  def debug(self,key):
    self.request('/debug', [key])

  def save(self,filename):
    self.request('/save', [filename])
    self.wait()

  def play(self,note):
    self.request('/play', [note + 40])

  def run(self,filename):
    self.set('output_filename', filename)
    self.start() # TODO: on host: unset 'batch_complete'
    self.wait()

  def reset(self):
    self.request('/reset', [])
  
  def pause(self):
    self.request('/pause', [])

  def end(self):
    self.request('/end', [])

  def generate(self, length, unit):
    self.request('/generate', [length, unit])
    self.wait()

  def wait(self):
    self.log("waiting...")
    self.server.serve_forever()
    self.log("ok!")
  
  def load_bars(self,filename,barcount):
    self.request('/load_bars', [filename, barcount])
    self.wait()

client = BatchClient(log, ip,  port,  port_in)
logfile = None
def start_model(modelname, checkpointname):
    logfile = open(path.join(outputdir, f'{modelname}.log'), 'w')
    modelname = modelname
    checkpointname = checkpoint
    log(f'Starting model: {modelname} ({checkpointname})')
    env = environ.copy()
    env['NOT_INTERACTIVE'] = '1'
    # server = subprocess.Popen([ './start.sh', model, checkpoint, 'batch' ], cwd=path.pardir, env=env)
    server = subprocess.Popen([ './start.sh', modelname, checkpointname, 'batch' ], env=env, stdout=logfile, stderr=logfile)
    # server.communicate()
    client.wait()
    client.set('debug_output', False)
    client.set('batch_mode', True)
    client.set('trigger_generate', 1)
    client.set('batch_unit', 'measures')
    client.set('debug_output', False)

def stop_model():
  if client: client.end()
  if server: server.wait()
  if logfile: logfile.close()


In [6]:
# EXPERIMENT BLOCK
try:
    log(f'Starting experiment suite: {outputdir}/{foldername}')
    df = pd.DataFrame([], columns=cols_gen)
    for dataset, models in evaluation_sets:
        if dataset is None: continue
        datapath = path.join(datasetdir, dataset)
        primerdir = datapath
        if not path.exists(datapath):
            print(f'Directory not found: {datapath}, skipping')
            continue

        primers = listdir(datapath)[:1] # FIXME: Quick Test
        # primers = listdir(datapath)

        for (model, checkpoint) in models:
            expname = 'setup'
            start_model(model, checkpoint)
            for i in range(iterations):
                for name, exp in experiments:
                    df += experiment(name, exp)
            stop_model()
    
    df.describe()

except KeyboardInterrupt:
  print("Terminating...")
  client.pause()
finally:
  stop_model()

print("Done!")

[batch:] Starting experiment suite: ./output/2021-05-17-12
[batch:setup] Starting model: polyphony_rnn (polyphony_rnn)
[batch:setup] waiting...
[batch:setup] ok!
[batch:setup] /set debug_output False
[batch:setup] /set batch_mode True
[batch:setup] /set trigger_generate 1
[batch:setup] /set batch_unit measures
[batch:setup] /set debug_output False
[batch:setup] /reset 
[batch:setup] /load_bars ./dataset/bach-chorales/260.mid 1
[batch:setup] waiting...
[batch:setup] ok!
[batch:setup] /generate 1 measures
[batch:setup] waiting...
[batch:setup] ok!
[batch:setup] /save 2021-05-17-12/generation-260.mid-1-1-bars-1
[batch:setup] waiting...
[batch:setup] ok!
[[None, None, '260.mid', '2021-05-17-12/generation-260.mid-1-1-bars-1', 1.4234930730017368, 1, 1]]
[batch:setup] /reset 
[batch:setup] /load_bars ./dataset/bach-chorales/260.mid 1
[batch:setup] waiting...
[batch:setup] ok!
[batch:setup] /generate 1 measures
[batch:setup] waiting...
[batch:setup] ok!
[batch:setup] /save 2021-05-17-12/genera

TypeError: join() argument must be str or bytes, not 'NoneType'